# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [6]:
# Remover colunas irrelevantes
df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'], inplace=True)

# Remover linhas com valores nulos em tempo_emprego (opcionalmente podemos imputar também)
df = df.dropna(subset=['tempo_emprego']).copy()

# Aplicar log na renda
df['log_renda'] = np.log(df['renda'])

# Fórmula inicial (com todas as variáveis explicativas)
formula = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

modelo_inicial = smf.ols(formula=formula, data=df).fit()
print(modelo_inicial.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 08 Apr 2025   Prob (F-statistic):               0.00
Time:                        15:33:27   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [9]:
import unidecode

# 1. Criar dummies
df_dummies = pd.get_dummies(df, drop_first=True)

# 2. Renomear colunas para remover acentos, espaços e caracteres especiais
df_dummies.columns = [
    unidecode.unidecode(col).replace(" ", "_").replace("-", "_")
    for col in df_dummies.columns
]

# 3. Remover variável menos significativa do modelo anterior
df_dummies = df_dummies.drop(columns=['educacao_Secundario'])

# 4. Montar fórmula (sem crase agora, pois os nomes estão limpos)
variaveis = " + ".join([col for col in df_dummies.columns if col != 'log_renda'])
formula2 = f"log_renda ~ {variaveis}"

# 5. Ajustar o novo modelo
modelo2 = smf.ols(formula=formula2, data=df_dummies).fit()
print(modelo2.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     817.1
Date:                Tue, 08 Apr 2025   Prob (F-statistic):               0.00
Time:                        15:35:26   Log-Likelihood:                -10424.
No. Observations:               12427   AIC:                         2.090e+04
Df Residuals:                   12402   BIC:                         2.108e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

O Modelo 2 apresentou melhora em todos os principais indicadores. 